In [2]:
import os
import pickle
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import LabelEncoder

# ================================
# Paths
# ================================
train_dir = r"C:\Users\sagni\Downloads\food spoilage prediction\archive (1)\dataset\Train"
save_dir = r"C:\Users\sagni\Downloads\food spoilage prediction"

# ================================
# Parameters
# ================================
img_size = 224
batch_size = 32
epochs = 10
learning_rate = 0.0001

# ================================
# Data Preprocessing
# ================================
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)

# Save label encoder
label_encoder = LabelEncoder()
label_encoder.fit(list(train_gen.class_indices.keys()))

label_encoder_path = os.path.join(save_dir, "label_encoder.pkl")
with open(label_encoder_path, "wb") as f:
    pickle.dump(label_encoder, f)
print(f"Label encoder saved to {label_encoder_path}")

# ================================
# Model
# ================================
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze base model for transfer learning

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(len(train_gen.class_indices), activation="softmax")
])

model.compile(optimizer=Adam(learning_rate=learning_rate),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# ================================
# Checkpoint
# ================================
checkpoint_path = os.path.join(save_dir, "best_model.h5")
checkpoint = ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1)

# ================================
# Training
# ================================
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[checkpoint]
)

# ================================
# Save Final Model in .h5
# ================================
final_model_path = os.path.join(save_dir, "food_spoilage_model.h5")
model.save(final_model_path)
print(f"Model saved to {final_model_path}")

# ================================
# Save training history for later
# ================================
history_path = os.path.join(save_dir, "training_history.pkl")
with open(history_path, "wb") as f:
    pickle.dump(history.history, f)
print(f"Training history saved to {history_path}")


Found 18901 images belonging to 18 classes.
Found 4718 images belonging to 18 classes.
Label encoder saved to C:\Users\sagni\Downloads\food spoilage prediction\label_encoder.pkl


C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.4131 - loss: 2.0001
Epoch 1: val_accuracy improved from -inf to 0.76812, saving model to C:\Users\sagni\Downloads\food spoilage prediction\best_model.h5


591/591 ━━━━━━━━━━━━━━━━━━━━ 365s 608ms/step - accuracy: 0.4134 - loss: 1.9990 - val_accuracy: 0.7681 - val_loss: 0.6957
Epoch 2/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 0.8187 - loss: 0.5749
Epoch 2: val_accuracy improved from 0.76812 to 0.80437, saving model to C:\Users\sagni\Downloads\food spoilage prediction\best_model.h5


591/591 ━━━━━━━━━━━━━━━━━━━━ 345s 584ms/step - accuracy: 0.8187 - loss: 0.5748 - val_accuracy: 0.8044 - val_loss: 0.5719
Epoch 3/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.8732 - loss: 0.3867
Epoch 3: val_accuracy improved from 0.80437 to 0.82556, saving model to C:\Users\sagni\Downloads\food spoilage prediction\best_model.h5


...

591/591 ━━━━━━━━━━━━━━━━━━━━ 344s 583ms/step - accuracy: 0.9093 - loss: 0.2662 - val_accuracy: 0.8444 - val_loss: 0.4812
Epoch 6/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.9226 - loss: 0.2390
Epoch 6: val_accuracy improved from 0.84443 to 0.85015, saving model to C:\Users\sagni\Downloads\food spoilage prediction\best_model.h5


591/591 ━━━━━━━━━━━━━━━━━━━━ 349s 590ms/step - accuracy: 0.9226 - loss: 0.2389 - val_accuracy: 0.8501 - val_loss: 0.4892
Epoch 7/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.9308 - loss: 0.2087
Epoch 7: val_accuracy improved from 0.85015 to 0.85418, saving model to C:\Users\sagni\Downloads\food spoilage prediction\best_model.h5


591/591 ━━━━━━━━━━━━━━━━━━━━ 347s 587ms/step - accuracy: 0.9308 - loss: 0.2087 - val_accuracy: 0.8542 - val_loss: 0.4818
Epoch 8/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9282 - loss: 0.2011
Epoch 8: val_accuracy improved from 0.85418 to 0.86414, saving model to C:\Users\sagni\Downloads\food spoilage prediction\best_model.h5


591/591 ━━━━━━━━━━━━━━━━━━━━ 347s 587ms/step - accuracy: 0.9282 - loss: 0.2011 - val_accuracy: 0.8641 - val_loss: 0.4978
Epoch 9/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.9353 - loss: 0.1819
Epoch 9: val_accuracy did not improve from 0.86414
591/591 ━━━━━━━━━━━━━━━━━━━━ 353s 597ms/step - accuracy: 0.9353 - loss: 0.1819 - val_accuracy: 0.8554 - val_loss: 0.4819
Epoch 10/10
591/591 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.9397 - loss: 0.1691
Epoch 10: val_accuracy did not improve from 0.86414
591/591 ━━━━━━━━━━━━━━━━━━━━ 356s 603ms/step - accuracy: 0.9397 - loss: 0.1691 - val_accuracy: 0.8550 - val_loss: 0.4878


Model saved to C:\Users\sagni\Downloads\food spoilage prediction\food_spoilage_model.h5
Training history saved to C:\Users\sagni\Downloads\food spoilage prediction\training_history.pkl
